# 라이브러리 import

In [24]:
!pip install celluloid

import os
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
import easydict
from tqdm.notebook import trange, tqdm
from torch.utils.data import DataLoader, Dataset
from celluloid import Camera
import pandas as pd
import pickle
from typing import List

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Data Import

In [25]:
train_df = pd.read_csv('/content/drive/MyDrive/LG Aimers 2기/데이터/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/LG Aimers 2기/데이터/test.csv')

In [26]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Columns: 2881 entries, PRODUCT_ID to X_2875
dtypes: float64(2876), int64(1), object(4)
memory usage: 13.1+ MB


In [27]:
train_df.head()

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,A_31,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,TRAIN_003,2,0.537325,2022-06-13 5:39,T050307,A_31,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,TRAIN_004,1,0.531590,2022-06-13 5:47,T050304,A_31,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN


In [28]:
train_x = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
train_y = train_df['Y_Class']

test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

Y_Class : 제품 품질 상태(Target) 

0 : 적정 기준 미달 (부적합)

1 : 적합

2 : 적정 기준 초과 (부적합)

In [29]:
## 결측 변수 비율 확인
(train_x.isnull().sum()/len(train_x)).plot.bar(figsize=(500, 100), colormap='Paired')

결측 비율이 높은 데이터는 컬럼 제거 필요....

나머지 결측 비율을 가진 데이터들은 한 시점 이전 데이터 이용하여 보간해준다.

In [30]:
df_info = pd.DataFrame(train_x.isnull().sum()/len(train_x) > 0.8)

In [31]:
df_info

,0
LINE,False
PRODUCT_CODE,False
X_1,False
X_2,False
X_3,False
...,...
X_2871,True
X_2872,True
X_2873,True
X_2874,True


In [32]:
list_dell = df_info[df_info[0]==True].index

In [33]:
type(list_dell)

pandas.core.indexes.base.Index

In [34]:
list_dell = list(list_dell)
print(list_dell)

['X_128', 'X_129', 'X_130', 'X_131', 'X_337', 'X_338', 'X_339', 'X_340', 'X_341', 'X_342', 'X_356', 'X_357', 'X_358', 'X_360', 'X_361', 'X_362', 'X_371', 'X_372', 'X_373', 'X_374', 'X_382', 'X_383', 'X_529', 'X_530', 'X_531', 'X_532', 'X_533', 'X_561', 'X_600', 'X_601', 'X_602', 'X_603', 'X_604', 'X_699', 'X_759', 'X_760', 'X_761', 'X_934', 'X_935', 'X_936', 'X_937', 'X_990', 'X_991', 'X_1009', 'X_1010', 'X_1011', 'X_1012', 'X_1167', 'X_1168', 'X_1169', 'X_1170', 'X_1171', 'X_1174', 'X_1175', 'X_1219', 'X_1274', 'X_1275', 'X_1276', 'X_1277', 'X_1278', 'X_1407', 'X_1523', 'X_1524', 'X_1525', 'X_1533', 'X_1535', 'X_1537', 'X_1540', 'X_1680', 'X_1681', 'X_1682', 'X_1683', 'X_1684', 'X_1685', 'X_1686', 'X_1687', 'X_1688', 'X_1689', 'X_1690', 'X_1691', 'X_1692', 'X_1693', 'X_1694', 'X_1695', 'X_1707', 'X_1708', 'X_1709', 'X_1710', 'X_1951', 'X_1952', 'X_1953', 'X_1954', 'X_1955', 'X_1956', 'X_1957', 'X_1958', 'X_1959', 'X_1960', 'X_1961', 'X_1962', 'X_1963', 'X_1964', 'X_1965', 'X_1966', 'X

In [35]:
train_x = train_x.drop(columns=list_dell)

test_x = test_x.drop(columns=list_dell)

In [36]:
train_x = train_x.fillna(method='ffill')
test_x = test_x.fillna(method='ffill')

In [37]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')

Done.


In [38]:
XGB = XGBClassifier(random_state=37).fit(train_x, train_y)

In [39]:
preds = XGB.predict(test_x)

In [40]:
submit = pd.read_csv('/content/drive/MyDrive/LG Aimers 2기/데이터/sample_submission.csv')


submit['Y_Class'] = preds

In [41]:
submit.to_csv('/content/drive/MyDrive/LG Aimers 2기/제출/baseline_submission3.csv', index=False)